In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.29/01


In [2]:
iOption0 = 0

In [3]:
isigma = 2
N_files = 1
N_centr = 5
N_hists = 2
N_layers = 6
N_ilyaers = 8
is_sigmalised = 0
iOption1 = 0
centralities = ["0-20%","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5]
layer_bins = [1,1,2,2,3,3,4,4,5,5,6,6]
if iOption0 == 1:
    N_layers = 6
    layer_bins = [1,1,2,2,3,3,4,4,5,5,6,6]
colors=[1,2,4,root.kGreen+2,root.kMagenta,3]
file_path="input/"
file_names="my-sixth_QA_19081_550runs"
hist_select_3D_names = ["d_dphi_hist","d_dthe_hist"]

In [4]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            #if icentr==4: hist_select_3D0[-1].Add(infile.Get(hist_select_3D_names[ihist]+f"_{icentr-1}"))
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/my-sixth_QA_19081_550runs0.root


In [5]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.7,1.0,1.4,2.0,2.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append(0)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)

pt =  array('d', [0.25, 0.35, 0.45, 0.6, 0.85, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.7, 1.0, 1.4, 2.0, 2.8, 4.9])


In [6]:
boards_tot = [[-0.04,0.04],[-0.04,0.04]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[0.01,0.03],[-0.03,-0.01]]
if iOption1==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]

In [7]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(icentr)+"/"+str(ilayer)+"/emc_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit[int(seed0%2)][0], boards_fit[int(seed0%2)][1])
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit2[int(ilayer%2)][0], boards_fit2[int(ilayer%2)][1])#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{icentr}_{ilayer}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(seed0%2)][0],boards_tot[int(seed0%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{icentr}_{ilayer}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(seed0%2)][0], boards_tot[int(seed0%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[int(seed0%2)][1])
        g_gaus2.SetParameter(1,0.)
        g_gaus2.SetParameter(2,0.003)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 1:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(seed0%2)][0]/10,boards_tot[int(seed0%2)][1]/10)
        total.SetParLimits(2,boards_fit[int(seed0%2)][1]/10,boards_tot[int(seed0%2)][1]/2)
        total.SetParLimits(4,boards_fit2[int(ilayer%2)][0], boards_fit2[int(ilayer%2)][1])
        if ilayer>1:
            total.SetParLimits(1,boards_tot[int(seed0%2)][0],boards_tot[int(seed0%2)][1])
            total.SetParLimits(4,-100,100)

        total.SetParLimits(5,0.0005,0.005)

        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        if iOption0 ==0:
            Mean_local.append(total.GetParameter(1))
            Sigma_local.append(total.GetParameter(2))
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        if fits_mean==0:
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
        else:
            c1.SaveAs( "output/d_delta_fits/" + m2_name + "upd.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [8]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+iOption1*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [9]:
import multiprocess as mp

Ntr = N_files

input_arr = [0,1]
if iOption0 == 1: input_arr = [0,1,2,3,4,5]
input_arr = [0,1,2,3,4,5]

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, input_arr, i, isigma) for i in range(Ntr)])
pool.close()

In [10]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [11]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
colors=[1,2,4,6,root.kGreen+2,root.kOrange]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.01512)
            fit_functions1[-1].SetParameter(1,0.00912)
            fit_functions1[-1].SetParLimits(1,0.001,0.018)
            fit_functions1[-1].SetParameter(2,-2.02)
            fit_functions1[-1].SetParLimits(2,-7.126,-0.023)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [12]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0., 0.03
    if iOption1==1: max, min = 4.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))
            #g_sigma_inter[icentr][ilayer].append(root.TGraphErrors(N_pt))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            #Format_Graph(g_sigma_inter[icentr][ilayer][-1], 21, 0, colors[ilayer], 3, colors[ilayer], 1, 1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","")
            #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            #g_sigma_inter[icentr][ilayer][-1].Draw('sameL')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")
            #legends[-1].AddEntry(fit_functions[icentr][ilayer],f"fit","l")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{iOption1}.png")

array('d', [0.005245757376735619, 0.003640799739768584, 0.002751917229286841, 0.0022539215930653337, 0.0020100118470120438, 0.0017009962151394783, 0.0015070961745554088, 0.0014194153387577801, 0.0014315355674638234]) array('d', [0.004763467393925919, 0.0032905155581304532, 0.0026558559667291797, 0.0022205307292002645, 0.0019015509649617105, 0.0016595598852865033, 0.0014789436685578269, 0.0013806160141135592, 0.0012970671867010464])


python ERROR: cannot open image file "output/Sigmas/sigma_000.png" for writing. Please check permissions.


In [13]:
print(f"params[{N_files}][{1}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    fit_functions_icentr = fit_functions_ifile[4]
    print("{")
    for fits in fit_functions_icentr:
        print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
    print("},") 
    print("},")      
print("};")        

params[1][1][6][3]={
{
{
{0.001221, 0.006044, -3.305693},
{0.001245, 0.00569, -3.315811},
{0.001645, 0.009011, -2.098423},
{0.001916, 0.009606, -2.568957},
{0.001897, 0.00912, -1.767405},
{0.002206, 0.011189, -2.307913},
},
},
};


In [14]:
print(f"params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

params[1][5][6][3]={
{
{
{0.00153, 0.010007, -4.329956},
{0.001496, 0.008956, -4.139011},
{0.001879, 0.011609, -2.630026},
{0.001771, 0.010259, -2.440109},
{0.00181, 0.009388, -1.715039},
{0.001773, 0.008405, -1.551142},
},
{
{0.001491, 0.008258, -3.92518},
{0.001484, 0.007885, -3.934144},
{0.001837, 0.010593, -2.447101},
{0.001907, 0.010912, -2.708126},
{0.002033, 0.010241, -1.987386},
{0.002029, 0.010228, -2.002456},
},
{
{0.001491, 0.007255, -3.751226},
{0.001441, 0.006783, -3.62288},
{0.001904, 0.010432, -2.525286},
{0.001896, 0.010381, -2.639697},
{0.002074, 0.010823, -2.125175},
{0.002042, 0.010977, -2.135211},
},
{
{0.001495, 0.008004, -4.021743},
{0.001574, 0.007515, -4.316167},
{0.001989, 0.010129, -2.562587},
{0.001983, 0.01015, -2.680598},
{0.002187, 0.011307, -2.273443},
{0.002043, 0.010457, -2.04727},
},
{
{0.001221, 0.006044, -3.305693},
{0.001245, 0.00569, -3.315811},
{0.001645, 0.009011, -2.098423},
{0.001916, 0.009606, -2.568957},
{0.001897, 0.00912, -1.767405},
{0.002

In [15]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            if iOption0 == 0:
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]/sqrt(x)+[2]/x",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.002*(0.5-ilayer))
                fit_functions1[-1].SetParameter(1,-0.004*(0.5-ilayer))
            if iOption0 == 1:
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]",0.5,5.0))
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [16]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3",f"c3",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.009, 0.009
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            #g_sigma_inter[icentr][ilayer].append(root.TGraphErrors(N_pt))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            #Format_Graph(g_sigma_inter[icentr][ilayer][-1], 21, 0, colors[ilayer], 3, colors[ilayer], 1, 1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","")
            #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            #g_sigma_inter[icentr][ilayer][-1].Draw('sameL')
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"layer {ilayer}","p")
            #legends[-1].AddEntry(fit_functions[icentr][ilayer],f"fit","l")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

python ERROR: cannot open image file "output/Sigmas/mean_000.png" for writing. Please check permissions.


In [17]:
print(f"params[{N_files}][{1}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_mean_functions:
    print("{")
    fit_functions_icentr = fit_functions_ifile[4]
    print("{")
    for fits in fit_functions_icentr:
        if iOption0 ==0: print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        else: print("{"+f"{round(fits.GetParameter(0),6)}"+"},")
    print("},") 
    print("},")      
print("};")        

params[1][1][6][3]={
{
{
{0.000422, -0.000956, 0.00111},
{-0.00017, 0.000711, -0.001062},
{0.001334, -0.002706, 0.00297},
{-0.000647, 0.001988, -0.002752},
{0.001475, -0.002978, 0.003609},
{-0.001147, 0.003132, -0.003703},
},
},
};


In [18]:
print(f"params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_mean_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

params[1][5][6][3]={
{
{
{0.001278, -0.002833, 0.001941},
{-0.000875, 0.002166, -0.001665},
{0.001274, -0.002568, 0.00267},
{-0.000386, 0.000966, -0.001949},
{0.000705, -0.001539, 0.002622},
{-0.000985, 0.002632, -0.003149},
},
{
{0.000684, -0.001598, 0.001444},
{-0.000434, 0.001182, -0.001275},
{0.000338, -0.001088, 0.002303},
{-0.000135, 0.000576, -0.002007},
{0.000305, -0.000922, 0.002651},
{0.0, 0.000623, -0.002457},
},
{
{0.000507, -0.001177, 0.001294},
{-2.7e-05, 0.000335, -0.000964},
{0.000334, -0.001139, 0.002461},
{-3.7e-05, 0.000435, -0.002103},
{0.000281, -0.000995, 0.002875},
{1.4e-05, 0.000653, -0.002665},
},
{
{0.000762, -0.001702, 0.001564},
{-0.000655, 0.001308, -0.001311},
{0.000515, -0.001519, 0.002656},
{-0.000355, 0.001171, -0.002458},
{0.000479, -0.001345, 0.003069},
{-0.000225, 0.00133, -0.003034},
},
{
{0.000422, -0.000956, 0.00111},
{-0.00017, 0.000711, -0.001062},
{0.001334, -0.002706, 0.00297},
{-0.000647, 0.001988, -0.002752},
{0.001475, -0.002978, 0.003609},